In [23]:
import oracledb
import csv
import os
import pandas as pd

In [24]:
import os

# Get the path to the password file
password_file = os.path.join(os.path.dirname('/Users/constah/Documents/Python/ADW/'), 'pw.txt')
# Open the password file
with open(password_file, 'r') as f:
    pw = f.read()


In [25]:
##Define connection
connection=oracledb.connect(
     user="",
     password=pw,
     dsn="db_testing_high",
     config_dir="/Users/constah/Documents/Python/Wallet_File/",
     wallet_location="/Users/constah/Documents/Python/Wallet_File/",
     wallet_password="password"
)
# Create a cursor
cursor = connection.cursor()

In [26]:
df=pd.read_csv('leads.csv')
df.head()

,id,status,contract_reference,state,county
0,3858,Completed,C46760013,5069,5117
1,4517,Completed,C53890018,5069,5117
2,68525,Completed,C700350010,5069,5117
3,4437,Completed,C53030011,5069,5117
4,65102,Completed,C667420012,5071,5118


### LOAD CONTRACT OFFERS

In [57]:

#Drop Table 
#cursor.execute("DROP TABLE CONTRACT_OFFERS")
# Create a table for the contract_offers dataset
cursor.execute("""
CREATE TABLE contract_offers (
  id number,
  name VARCHAR2(255),
  type VARCHAR2(255),
  total_value FLOAT
)
""")
# Commit the table creation
connection.commit()

# Read the data from the CSV file into a Python list, excluding the header column.
with open('contract_offers.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    data = []
    for row in csv_reader:
        data.append(row)

# Use the cursor object to insert the data into the Oracle database table.
sql = 
"""
insert into contract_offers (id, name, type, total_value)values (:1, :2, :3, :4)
"""
cursor.executemany(sql, data)

# Commit the changes to the database.
connection.commit()

# Close the connection to the database.
connection.close()


In [75]:
df.dtypes

reference                  object
status                     object
start_date                 object
offer_id                    int64
lead_id                     int64
cumulative_amount_paid    float64
nominal_contract_value    float64
dtype: object

### LOAD CONTRACTS

In [85]:

# Create a table for the contracts dataset
cursor.execute("""
CREATE TABLE contracts (
  reference VARCHAR2(255),
  status VARCHAR2(255),
  start_date VARCHAR2(255),
  offer_id number,
  lead_id number,
  cumulative_amount_paid float,
  nominal_contract_value float  
)
""")
# Commit the table creation
connection.commit()

# Read the data from the CSV file into a Python list, excluding the header column.
with open('contracts.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    data = []
    for row in csv_reader:
        data.append(row)

# Use the cursor object to insert the data into the Oracle database table.
sql = """
insert into contracts 
(reference, status, start_date, offer_id,lead_id,cumulative_amount_paid,nominal_contract_value)
values (:1, :2, :3, :4,:5,:6,:7)
"""
cursor.executemany(sql, data)

# Commit the changes to the database.
connection.commit()

# Close the connection to the database.
connection.close()




### LOAD PAYMENTS

In [87]:
df.dtypes


id                      int64
contract_reference     object
payment_date           object
type                   object
amount_paid           float64
dtype: object

In [90]:

# Create a table for the contracts dataset
cursor.execute("""
CREATE TABLE contract_payments (
  id number,
  contract_reference VARCHAR2(255),
  payment_date VARCHAR2(255),
  type VARCHAR2(255),
  amount_paid float 
)
""")
# Commit the table creation
connection.commit()

# Read the data from the CSV file into a Python list, excluding the header column.
with open('contract_payments.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    data = []
    for row in csv_reader:
        data.append(row)

# Use the cursor object to insert the data into the Oracle database table.
sql = """
insert into contract_payments (id, contract_reference, payment_date, type,amount_paid)
values (:1, :2, :3, :4,:5)
"""
cursor.executemany(sql, data)

# Commit the changes to the database.
connection.commit()

# Close the connection to the database.
connection.close()


### LOAD LEADS

In [21]:
df.dtypes

id                     int64
status                object
contract_reference    object
state                  int64
county                 int64
dtype: object

In [22]:
# Create a table for the contracts dataset
cursor.execute("""
CREATE TABLE leads (
  id number,
  status VARCHAR2(255),
  contract_reference VARCHAR2(255),
  state number,
  county number 
)
""")
# Commit the table creation
connection.commit()

# Read the data from the CSV file into a Python list, excluding the header column.
with open('leads.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    data = []
    for row in csv_reader:
        data.append(row)

# Use the cursor object to insert the data into the Oracle database table.
sql = """
insert into leads (id, status, contract_reference, state,county)
values (:1, :2, :3, :4,:5)
"""
cursor.executemany(sql, data)

# Commit the changes to the database.
connection.commit()

# Close the connection to the database.
connection.close()

### JOIN TABLES AND WRITE OUTPUT TO CSV

In [27]:
output="""
SELECT
    c.reference                                                          AS contract_reference,
    c.status,
    CASE
        WHEN upper(co.name) LIKE '%CASH%' THEN
            'CASH SALE'
        WHEN upper(name) LIKE '%GROUP%' THEN
            'GROUP LOAN'
        WHEN upper(name) LIKE '%INDIVIDUAL%' THEN
            'INDIVIDUAL'
        ELSE
            'OTHER'
    END                                                                  loan_type,
    trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 180 end_date,
    CASE
        WHEN upper(co.name) LIKE '%GROUP%' THEN
            trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 210
        WHEN name = 'Paygo Loan' THEN
            trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 30
        WHEN upper(name) LIKE '%INDIVIDUAL%' THEN
            trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 60
    END                                                                  maturity_date,
    to_char(
        CASE
            WHEN upper(co.name) LIKE '%GROUP%' THEN
                trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 210
            WHEN name = 'Paygo Loan' THEN
                trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 30
            WHEN upper(name) LIKE '%INDIVIDUAL%' THEN
                trunc(TO_TIMESTAMP(start_date, 'YYYY-MM-DD"T"HH24:MI:SS.FF6')) + 60
        END,
        'Q')                                                         quarter,
    c.cumulative_amount_paid,
    c.nominal_contract_value,
    l.state,
    l.county
FROM
    contracts       c
    LEFT JOIN contract_offers co ON c.offer_id = co.id
    LEFT JOIN leads           l ON l.id = c.lead_id
                         AND c.reference = l.contract_reference
"""

cursor.execute(output)

# Fetch all the rows
rows = cursor.fetchall()

# Get the column names
columns = [desc[0] for desc in cursor.description]

# Create a data frame
df = pd.DataFrame(rows, columns=columns)

# Close the cursor and connection
cursor.close()
connection.close()


In [29]:
# Print the data frame
df.head(10)


,CONTRACT_REFERENCE,STATUS,LOAN_TYPE,END_DATE,MATURITY_DATE,QUARTER,CUMULATIVE_AMOUNT_PAID,NOMINAL_CONTRACT_VALUE,STATE,COUNTY
0,C37840014,Active,INDIVIDUAL,2022-02-04,2021-10-07,4,61.00,119.00,5071,5118
1,C41540014,Active,INDIVIDUAL,2022-01-26,2021-09-28,3,3355.43,6046.43,5071,5118
2,C41870015,Active,INDIVIDUAL,2022-02-03,2021-10-06,4,3135.00,5678.00,5071,5118
3,C42500017,Active,INDIVIDUAL,2022-01-30,2021-10-02,4,4286.00,5844.00,5071,5118
4,C42610014,Active,INDIVIDUAL,2022-02-09,2021-10-12,4,5798.00,7736.00,5069,5117
5,C42710012,Active,INDIVIDUAL,2022-02-05,2021-10-08,4,10258.00,10262.00,5069,5117
6,C42730010,Active,INDIVIDUAL,2022-02-13,2021-10-16,4,5528.00,10283.00,5069,5117
7,C42790014,Active,INDIVIDUAL,2022-02-02,2021-10-05,4,5898.00,7398.00,5069,5117
8,C42800011,Active,INDIVIDUAL,2022-02-05,2021-10-08,4,10158.00,10162.00,5069,5117
9,C43090018,Active,INDIVIDUAL,2022-02-13,2021-10-16,4,5628.00,10262.00,5069,5117


### Write Dataframe Output to CSV

In [30]:
# Write the DataFrame to a CSV file
df.to_csv('Tupande Output.csv', index=False)